In [ ]:
#| default_exp discriminator
from nbdev.showdoc import *
import numpy as np
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import torch
import torch_geometric
%load_ext autoreload
%autoreload 2

# Scattering Discriminator
> Classify graphs as belonging to our dataset (or not) using a scattering network

FEARFRED's generator creates and samples from a flow field. The Flashlight Kernel converts these samples into a directed graph. At last, we arrive at the final piece in our architecture: a GNN to discriminate between graphs coming from our dataset and graphs coming from the flow field. 

The trick here is to preserve the chain of differentiability between the generator and discriminator. This makes any translation of the affinity matrix from the flashlight kernel into a PyTorch Geometric format problematic -- this translation isn't differentiable! It's better to stay as close to the affinity matrix as possible.

To do this, we adopt a *learnable scattering network*, LEGSNet, which operates entirely on the affinity matrix of a graph. This instatiation of LEGSNet uses the scattering transform as our *witness function*, paired with a simple feedforward classifier to make the final call on in-group membership.

# Fixed LEGS Module
Adapted from Alex Tong's LEGSNet, this module implements a vanilla scattering network on an adjacency matrix and degree information. In this simpler discriminator, there are no learnable parameters in the scattering transform -- only in the classifier.

In [ ]:
#| export
def scatter_moments(data, moments_returned=2):
    # given a matrix of node features of shape n_nodes x n_features, 
    # returns a vector of length moments_returned x n_features comprising the mean and 
    # variance of each feature
    statistical_moments = {"mean": torch.zeros(0)}
    if moments_returned >= 2:
        statistical_moments["variance"] = torch.zeros(0)
    if moments_returned >= 3:
        statistical_moments["skew"] = torch.zeros(0)
    if moments_returned >= 4:
        statistical_moments["kurtosis"] = torch.zeros(0)
    def m(i):  # ith moment, computed with derivation data
        return torch.mean(deviation_data ** i, axis=1)
    mean = torch.mean(data, dim=1, keepdim=True)
    if moments_returned >= 1:
        statistical_moments["mean"] = torch.cat(
            (statistical_moments["mean"], mean.T), dim=0
        )
    deviation_data = data - mean
    # variance: difference of u and u mean, squared element wise, summed and divided by n-1
    variance = m(2)
    if moments_returned >= 2:
        statistical_moments["variance"] = torch.cat(
            (statistical_moments["variance"], variance[None, ...]), dim=0
        )

    # skew: 3rd moment divided by cubed standard deviation (sd = sqrt variance), with correction for division by zero (inf -> 0)
    skew = m(3) / (variance ** (3 / 2))
    skew[
        skew > 1000000000000000
    ] = 0  # multivalued tensor division by zero produces inf
    skew[
        skew != skew
    ] = 0  # single valued division by 0 produces nan. In both cases we replace with 0.
    if moments_returned >= 3:
        statistical_moments["skew"] = torch.cat(
            (statistical_moments["skew"], skew[None, ...]), dim=0
        )

    # kurtosis: fourth moment, divided by variance squared. Using Fischer's definition to subtract 3 (default in scipy)
    kurtosis = m(4) / (variance ** 2) - 3
    kurtosis[kurtosis > 1000000000000000] = -3
    kurtosis[kurtosis != kurtosis] = -3
    if moments_returned >= 4:
        statistical_moments["kurtosis"] = torch.cat(
            (statistical_moments["kurtosis"], kurtosis[None, ...]), dim=0
        )
    # Concatenate into one tensor (alex)
    # statistical_moments = torch.cat([v for k,v in statistical_moments.items()], axis=1)
    statistical_moments = torch.cat([statistical_moments['mean'],statistical_moments['variance']],axis=1)
    return statistical_moments

In [ ]:
data = torch.rand(5,3)
print("data is ",data)
scatter_moments(data,moments_returned=4)

data is  tensor([[0.2874, 0.3005, 0.7480],
        [0.7218, 0.2515, 0.1180],
        [0.7301, 0.9778, 0.3636],
        [0.1182, 0.1867, 0.6095],
        [0.7845, 0.1892, 0.8575]])


tensor([[0.4453, 0.3638, 0.6905, 0.3048, 0.6104, 0.0458, 0.0671, 0.0637, 0.0472,
         0.0896]])

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
class ScatteringDiscriminator(nn.Module):
    """
    Given an affinity matrix and associated node features, classifies: 
    does this belong to the true dataset?
    The number of nodes per graph must be constant.
    """
    def __init__(self, 
                 n_nodes, # number of nodes in inputted graphs
                 n_features # number of features per node
                ):
        super().__init__()
        self.n_nodes = n_nodes 
        self.n_features = n_features
        self.n_scattering_moments = 2
        self.n_scattering_features = 11 * self.n_scattering_moments * self.n_features
        self.classifier_l1 = nn.Linear(self.n_scattering_features, 128)
        self.classifier_l2 = nn.Linear(128, 128)
        self.classifier_l3 = nn.Linear(128, 128)
        self.classifier_l4 = nn.Linear(128, 128)
        self.classifier_l5 = nn.Linear(128, 64)
        self.classifier_l6 = nn.Linear(64, 32)
        self.classifier_l7 = nn.Linear(32, 16)
        self.classifier_l8 = nn.Linear(16, 1)
        
    def diffuse(self,P, node_features):
        # diffuses node features using diffusion matrix
        # diffusion is left multiplication, equivalent to right multiplication by transpose
        node_features_transposed = node_features.permute(*torch.arange(node_features.ndim - 1, -1, -1))
        left_multiplication = node_features_transposed @ P
        transposed_left_mult = left_multiplication.permute(*torch.arange(left_multiplication.ndim - 1, -1, -1))

        return transposed_left_mult
        # return P.T @ node_features
    
    def feng_filters(self):
        tmp = np.arange(16).reshape(4,4)
        results = [4]
        for i in range(2, 4):
            for j in range(0, i):
                results.append(4*i+j)
        return results
    
    def scattering_transform(self, A, node_features):
        # Given an affinity matrix and node features, performs a wavelet transform
        # Adapted from Alex Tong's LEGSNet code
        # construct diffusion matrix
        D = torch.sum(A,axis=1)
        P = 1/2*torch.eye(len(A)) + 1/2*(A / D[:,None]) # TODO: ensure no self loops in A
        # TODO: Could make laziness learnable.
        # diffuse 16 times
        avgs = [node_features[:,:,None]]
        for i in range(16):
            avgs.append(self.diffuse(P, avgs[i]))
        # diffusion wavelets at various levels
        filter1 = avgs[1] - avgs[2]
        filter2 = avgs[2] - avgs[4]
        filter3 = avgs[4] - avgs[8]
        filter4 = avgs[8] - avgs[16]
        # print('filter shape',filter4.shape)
        # print('avg shape',avgs[1].shape)
        s0 = avgs[0]
        # the all-important nonlinearity
        s1 = torch.abs(torch.cat([filter1, filter2, filter3, filter4], dim=-1))
        # s1 is the set of scattering features for the graph
        # print('s1 shape is ',s1.shape)
        # repeat a second time
        avgs = [s1]
        for i in range(16):
            avgs.append(self.diffuse(P, avgs[i]))
        filter1 = avgs[1] - avgs[2]
        filter2 = avgs[2] - avgs[4]
        filter3 = avgs[4] - avgs[8]
        filter4 = avgs[8] - avgs[16]
        s2 = torch.abs(torch.cat([filter1, filter2, filter3, filter4], dim=-1))
        # print("s2 fresh",s2.shape)
        s2_reshaped = torch.reshape(s2, (-1, self.n_features, 4))
        # print("s2 after first reshape",s2.shape)
        s2 = torch.reshape(torch.transpose(s2_reshaped, 1, 2), (-1, 16, self.n_features))
        # print("s2 after second reshape",s2.shape)
        s2 = s2[:, self.feng_filters()]
        # print("s2 after feng filters",s2.shape)
        
        x = torch.cat([s0, s1], dim=2)
        # print("x shape merged s0 and s1",x.shape)
        x = torch.transpose(x, 1, 2)
        # print("x shape transposed",x.shape)
        # print("s2 shape",s2.shape)
        x = torch.cat([x, s2], dim=1)
        
        # print("x shape into scatter moments",x.shape)
        # collapse features
        x = x.reshape(11*self.n_features,self.n_nodes)
        # print("x reshaped",x.shape)

        x = scatter_moments(x,moments_returned = self.n_scattering_moments)
        # print("x shape out of scatter moments",x.shape)

        return x
    
    def classifier(self,scattering_features):
        x = self.classifier_l1(scattering_features)
        x = F.gelu(x)
        x = self.classifier_l2(x)
        x = F.gelu(x)
        x = self.classifier_l3(x)
        x = F.gelu(x)
        x = self.classifier_l4(x)
        x = F.gelu(x)
        x = self.classifier_l5(x)
        x = F.gelu(x)
        x = self.classifier_l6(x)
        x = F.gelu(x)
        x = self.classifier_l7(x)
        x = F.gelu(x)
        x = self.classifier_l8(x)
        return x
        
    def forward(self,A,node_features):
        x = self.scattering_transform(A,node_features)
        y = self.classifier(x)
        return y        

## Testing

In [ ]:
SD = ScatteringDiscriminator(n_nodes=42,n_features=2)

In [ ]:
A = torch.rand(42,42)
x = torch.ones(42,2)
SD(A,x)

tensor([[0.2413, 0.0148]], grad_fn=<AddmmBackward0>)